# STEP 1: PREPARE THE DATASET

* get the dataset from MS Marco 
* extract queries and documents 
* generate triplets of queries, relevant(positive) documents, and irrelevant (negative) documents
* tokenise your generated data

In [33]:
from datasets import load_dataset

df_hn = load_dataset("cocoritzy/week_2_triplet_dataset_hard_negatives")
df_sn = load_dataset("cocoritzy/week_2_triplet_dataset_soft_negatives")
# dataset = load_dataset("cocoritzy/week_2_triplet_dataset_hard_negatives", split="train[:10%]") # 10% of the datab
df_hn = df_hn["train"].to_pandas()
df_sn = df_sn["train"].to_pandas()

In [ ]:
df_hn.head()

In [35]:
# Get max length of all queries and passages because you neeed embedding dimensions to be the same

def get_max_length(df, column_name):
    lengths = []
    for text in df[column_name]:
        lengths.append(len(text))

    print(f"max length in {column_name}:", max(lengths))
    return 


In [ ]:
# Get max length in hn dataset
hn_negatives_length = get_max_length(df_hn, "negative_passage")
hn_positives_length = get_max_length(df_hn, "positive_passage")
hn_query_length = get_max_length(df_hn, "query")

In [ ]:
# Get max length in sn dataset
sn_negatives_length = get_max_length(df_sn, "negative_passage")
sn_positives_length = get_max_length(df_sn, "positive_passage")
sn_query_length = get_max_length(df_sn, "query")


This is the dataset you'll be using for training

In [ ]:
# Construct triplet datasets
triplet_hn = df_hn[["query", "positive_passage", "negative_passage"]]
triplet_sn = df_sn[["query", "positive_passage", "negative_passage"]]
triplet_hn.head()

In [39]:
# Combine all columns into a single string
marco_text = ' '.join(triplet_hn["query"] + " " + triplet_hn["positive_passage"] + " " + triplet_hn["negative_passage"])


In [40]:
# Tokenize the data 
import re
from collections import Counter

def tokenize_text(text):
    
    
    # remove punctuation, number, and non-alphabetic characters
    remove_punctuation = re.sub(r'[^\w\s]', '', text)
    remove_numbers = re.sub(r'\d+', '', remove_punctuation)
    

    lower_case_words = remove_numbers.lower()
    # Split by whitespace and filter out empty strings
    words = [word for word in lower_case_words.split() if word]

    top_k = 30000
    word_counts = Counter(words)
    vocab = dict(word_counts.most_common(top_k))
    word_to_id = {word: i for i, word in enumerate(vocab.keys())}
    id_to_word = {i: word for i, word in enumerate(vocab.keys())}

    # Debugging: Print the unique vocabulary
    
    print("Vocabulary size:", len(vocab))

    return vocab, word_to_id, id_to_word



In [ ]:
vocab, word_to_id, id_to_word = tokenize_text(marco_text)
print(word_to_id)

# Step 2: Load in a pretrained word2vec
* Coline's word2vec model embeddings to embed the query and document text

In [ ]:
# First make sure the GPU is being used 

import torch 
import torch.nn as nn
import torch.nn.functional as F
from huggingface_hub import hf_hub_download
from model import CBOW

if torch.cuda.is_available():
   print(f"GPU: {torch.cuda.get_device_name(0)}")
   print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
   # Enable cuDNN auto-tuner
   torch.backends.cudnn.benchmark = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Print that the GPU is being used
print(f"Using device: {device}")

In [43]:
# model with text8 only
model_path = hf_hub_download(repo_id="cocoritzy/cbow-upvotes_model", filename="cbow_model.pt")


In [ ]:
# Retrieve checkpoint 
checkpoint = torch.load(model_path, map_location=device) #A checkpoint is a file that saves the state of your model
token_to_index = checkpoint["token_to_index"]
embedding_dim= checkpoint["embedding_dim"]
vocab_size = len(token_to_index)  # fill in actual size

print(f"Vocabulary size: {vocab_size}")
print(f"Embedding dimension: {embedding_dim}")



In [ ]:
# Initialize the model
model = CBOW(vocab_size, embedding_dim).to(device)

# Load the model parameters from the checkpoint
model.load_state_dict(checkpoint["model_state_dict"])
model.eval() # contains the trained weights of the model 



In [ ]:
pretrained_weights = model.embeddings.weight.data.clone() # Get the actual matrix of the pretrained vectors (the weights)

# Quick sanity check
print(type(model.embeddings))  # nn.Embedding -> shows you the whole embedding layer
print(model.embeddings.weight.shape)  # torch.Size([vocab_size, embedding_dim])



# STEP 4: CREATE NEW EMBEDDINGS FOR THIS DATASET
Now we should align the pretrained embeddings with my new word_to_id. We will do this by: 
* create an empty embedding
* then filling the vectors from CBOW
* if the word is not found in CBOW it gives a random vector

In [47]:
import torch

new_vocab_size = len(word_to_id)  # Your 30k vocab size
embedding_dim = pretrained_weights.shape[1]

# Fill with zeros
embedding_matrix = torch.zeros((new_vocab_size, embedding_dim))


In [48]:
for word, new_idx in word_to_id.items():
    if word in token_to_index:
        old_idx = token_to_index[word]
        embedding_matrix[new_idx] = pretrained_weights[old_idx]
    else:
        # Word not found in CBOW — random vector
        embedding_matrix[new_idx] = torch.randn(embedding_dim)


sanity checker ⬇️

In [ ]:
# Check the embedding matrix
print(embedding_matrix.shape)
print(embedding_matrix[0])

# Look at a random word in the embedding matrix
print(id_to_word[100])
print(embedding_matrix[100])

In [ ]:
# Basic similarity check
def cosine_similarity(vec1, vec2):
    return torch.dot(vec1, vec2) / (torch.norm(vec1) * torch.norm(vec2))

word1, word2 = 'takeoff', 'airplane'  # Replace with actual words
if word1 in word_to_id and word2 in word_to_id:
    idx1, idx2 = word_to_id[word1], word_to_id[word2]
    similarity = cosine_similarity(embedding_matrix[idx1], embedding_matrix[idx2])
    print(f"Cosine similarity between {word1} and {word2}: {similarity}")

Looks sensible enough to proceed with creating the embedding layer

In [ ]:
embedding_layer = nn.Embedding(30000, 100)
embedding_layer.weight = nn.Parameter(embedding_matrix)
embedding_layer.weight.requires_grad = False

# Test the embedding layer
test_word = 'takeoff'
test_idx = word_to_id[test_word]
print(embedding_layer(torch.tensor([test_idx])).squeeze().tolist())

# STEP 5: USE EMBEDDINGS IN THE TWO TOWERS AND SET UP ALL THE CLASSES AND FUNCTIONS
* remember you have two datasets: triplet_hn (hard negatives) and triple_sn (soft negatives)
* Use the embedding layer to embed the queries and documents
* for this, you are using __average pooling__ (not a RNN)


Define the model architecture. Both towers should return a pooled embedding for the query and document

In [52]:
# First define the model architecture aka the Two Towers with average pooling
class QueryTower(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embedding = embedding_layer

    def forward(self, query_tokens):
        # Embed the query tokens
        embedded = self.embedding(query_tokens)
        # Average the embeddings (for average pooling)
        avg_query_embedding = torch.mean(embedded, dim=1)

        return avg_query_embedding
    
class DocumentTower(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embedding = embedding_layer
        
    def forward(self, doc_tokens):
        # Embed the document tokens
        embedded = self.embedding(doc_tokens)
        # Average the embeddings (for average pooling)
        avg_doc_embedding = torch.mean(embedded, dim=1)

        return avg_doc_embedding



Create a distance function based on cosine similarity for the query and documents

In [53]:
# Create a distance function to compare the query and document embeddings
# This function computes the similarity between the query and document embeddings, and return the top 5 most similar documents
def compute_similarity(query_embedding, doc_embedding, k=5):

    # Compute the cosine similarity between the query and document embeddings
    similarity = F.cosine_similarity(query_embedding, doc_embedding)
    
    # Get the top k most similar documents
    top_k_values, top_k_indices = torch.topk(similarity, k)
    return top_k_values, top_k_indices

Create a Dataset class to make sure the dataset is loaded in properly

In [54]:
from torch.utils.data import Dataset, DataLoader

# Create a dataset class for the triplet
class TripletDataset(Dataset):
    def __init__(self, all_queries, all_positive_passages, all_negative_passages, word_to_id):
        """
        Args:
            queries (list of str): List of query texts.
            positive_passages (list of str): List of positive passage texts.
            negative_passages (list of str): List of negative passage texts.
            word_to_id (dict): Dictionary mapping words to their indices.
        """
        self.queries = all_queries
        self.positive_passages = all_positive_passages
        self.negative_passages = all_negative_passages
        self.word_to_id = word_to_id

    def __len__(self):
        # return the total number of samples
        return len(self.queries)
    
    def tokenize_text(self, text):
        # Remove punctuation, numbers, and convert to lowercase
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        text = re.sub(r'\d+', '', text)      # Remove numbers
        text = text.lower()                  # Convert to lowercase
        tokens = [word for word in text.split() if word]  # Split and remove empty tokens
        return tokens
    
    def __getitem__(self, idx):
        # Get the query, positive passage, and negative passage for the given index
        query = self.queries[idx]
        positive_passage = self.positive_passages[idx]
        negative_passage = self.negative_passages[idx]

        # Tokenize the query and passages
        query_tokens = self.tokenize_text(query)
        positive_tokens = self.tokenize_text(positive_passage)
        negative_tokens = self.tokenize_text(negative_passage)

        # Conver to word ids
        query_indices = torch.tensor([self.word_to_id[word] for word in query_tokens if word in self.word_to_id])
        positive_indices = torch.tensor([self.word_to_id[word] for word in positive_tokens if word in self.word_to_id])
        negative_indices = torch.tensor([self.word_to_id[word] for word in negative_tokens if word in self.word_to_id])

        return query_indices, positive_indices, negative_indices
    

In [56]:

def pad_to_length(tensor, length, padding_value=0):
    # Calculate the padding size
    pad_size = length - tensor.size(0)
    if pad_size > 0:
        # Pad the tensor to the specified length
        return F.pad(tensor, (0, pad_size), value=padding_value)
    else:
        # If the tensor is already the desired length or longer, truncate it
        return tensor[:length]

def collate_fn(batch):
    # Separate the batch into queries, positive passages, and negative passages
    queries, positives, negatives = zip(*batch)
    
    # Define the fixed lengths (found from earlier in the notebook)
    max_query_length = 144
    max_positive_length = 1167
    max_negative_length = 1039
    
    # Pad sequences to the fixed lengths
    queries_padded = torch.stack([pad_to_length(q, max_query_length) for q in queries])
    positives_padded = torch.stack([pad_to_length(p, max_positive_length) for p in positives])
    negatives_padded = torch.stack([pad_to_length(n, max_negative_length) for n in negatives])
    
    return queries_padded, positives_padded, negatives_padded

In [59]:
# Define the Triplet Loss Function 

class TripletLoss(nn.Module):
    def __init__(self, margin):
        super(TripletLoss, self).__init__()
        self.margin = margin
        self.loss_fn = nn.TripletMarginLoss(margin=self.margin)

    def forward(self, query_embedding, positive_embedding, negative_embedding):
        return self.loss_fn(query_embedding, positive_embedding, negative_embedding)



CHEEKY TESTING ZONE ⬇️

In [ ]:
## cheeky test before we move on 

triplet_dataset = TripletDataset(triplet_hn["query"], triplet_hn["positive_passage"], triplet_hn["negative_passage"], word_to_id)

# Create DataLoader with the collate function
dataset_loader = DataLoader(triplet_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

query_tower = QueryTower(30000, 100)
doc_tower = DocumentTower(30000, 100)

# Run the model
for query_indices, positive_indices, negative_indices in dataset_loader:
    query_embedding = query_tower(query_indices)
    positive_embedding = doc_tower(positive_indices)
    negative_embedding = doc_tower(negative_indices)

    print("Query Embedding:", query_embedding)
    print("Positive Embedding:", positive_embedding)
    print("Negative Embedding:", negative_embedding)

The initial cheeky test showed that my tensors within queries and documents are different sizes. This is a problem because the torch.stack operation expects tensors in the batch to have the same size. So we'll fix it by padding the tensors based on the max lengths of the query and documents found in the earlier cells. 

# STEP 6: SETUP MODEL TRAINING

In [ ]:
import datetime
import wandb
wandb.login()

# Initialize settings
torch.manual_seed(42)

timestamp = datetime.datetime.now().strftime('%Y_%m_%d__%H_%M_%S')

In [ ]:
# Initialize Weights & Biases
wandb.init(project="two_tower_model", 
           entity="evelyntants",
           name=f"{timestamp}",
           config={
               # Model parameters
               "embedding_dim": 100,
               "vocab_size": 30000,
                
                # Training parameters
                "batch_size": 128,
                "learning_rate": 0.003,
                "num_epochs": 5,
                "train_split": 0.7,
                
                # Optimizer parameters
                "weight_decay": 1e-5,
                
                # DataLoader parameters
                "num_workers": 4,

                # Triplet Loss margin
                "triplet_margin": 1.0
            }
)